In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [62]:
df = pd.read_excel('catalog1.xlsx')

In [63]:
df.head()

,Артикул,Категория,Название
0,272886054,Electrical cables,"ОРЕОЛ Шнур ШВВП 2х0,75 мм ГОСТ 50 м черный 00-..."
1,267791061,Canteen spoons,"Ложка столовая «Силуэт», длина 20 см, толщина ..."
2,267791075,Constructors,"Конструктор «Мини-кафе», 114 деталей"
3,267791402,Sports balls,"Волейбольный мяч ONLYTOP, машинная сшивка, 18 ..."
4,267791496,Gift boxes,"Коробка подарочная складная ""Лютик"" 18,5 х 10 ..."


In [64]:
df['Name'] = df['Категория'].str.lower().str.cat(df['Название'].str.lower(), sep=' ')
df.head()


,Артикул,Категория,Название,Name
0,272886054,Electrical cables,"ОРЕОЛ Шнур ШВВП 2х0,75 мм ГОСТ 50 м черный 00-...","electrical cables ореол шнур шввп 2х0,75 мм го..."
1,267791061,Canteen spoons,"Ложка столовая «Силуэт», длина 20 см, толщина ...","canteen spoons ложка столовая «силуэт», длина ..."
2,267791075,Constructors,"Конструктор «Мини-кафе», 114 деталей","constructors конструктор «мини-кафе», 114 деталей"
3,267791402,Sports balls,"Волейбольный мяч ONLYTOP, машинная сшивка, 18 ...","sports balls волейбольный мяч onlytop, машинна..."
4,267791496,Gift boxes,"Коробка подарочная складная ""Лютик"" 18,5 х 10 ...","gift boxes коробка подарочная складная ""лютик""..."


In [65]:
documents = df['Name']
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)


In [66]:
print("Названия признаков (слов):")
print(vectorizer.get_feature_names_out())

Названия признаков (слов):
['00' '000' '0000' ... 'ёршиком' 'ї1' 'ї2']


In [67]:
n_neighbors = 3
model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute') 

In [68]:
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=3)

In [69]:
distances, indices = model.kneighbors(tfidf_matrix, n_neighbors=51)  # 51 сосед

In [70]:
recommendations = {}
for i, neighbor_indices in enumerate(indices):
    # Фильтр: удалим сам товар (предполагаем, что он входит первым, либо найдём его индекс)
    filtered = [idx for idx in neighbor_indices if idx != i]
    # берем первые 50
    recommendations[df.iloc[i]['Артикул']] = df.iloc[filtered[:50]]['Артикул'].tolist()

In [71]:
print(recommendations[tuple(recommendations.keys())[0]])
tuple(recommendations.keys())[0]

[265404872, 265404833, 256540307, 263100993, 263098338, 272847444, 262538633, 274019886, 262538598, 256191715, 256161136, 256161130, 256690308, 263100535, 272848515, 256592696, 256161162, 260495693, 256540490, 274019896, 272847448, 255845717, 256581724, 256605926, 256894477, 256177240, 257742276, 256540278, 256161160, 256540314, 265404844, 272152595, 273401485, 256540277, 256581788, 256202945, 256581769, 267381506, 256581774, 255961169, 255845636, 262538620, 256171247, 256194032, 256194047, 256194050, 256194040, 256194041, 256203010, 267076898]


np.int64(272886054)

In [72]:
new_df = pd.DataFrame({
    'A': [],
    'B': []
})
with open('catalog1_done.xlsx', 'a') as f:
    for i in range(len(recommendations.keys())):
        for j in range(len(recommendations[tuple(recommendations.keys())[i]])):
            new_row = f'{tuple(recommendations.keys())[i]}, {recommendations[tuple(recommendations.keys())[i]][j]}\n'
            f.write(new_row)

KeyboardInterrupt: 